In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2
from matplotlib import patches
import matplotlib.pyplot as plt

# Import dataset and clean it
df1= pd.read_csv('./data/MT_anomaly_detection.csv', sep=",", decimal='.' )
df1.head()

df = df1[['out', 'in','coil', 'state']]
df = df.dropna()
df = df.to_numpy()

In [ ]:

# Covariance matrix
covariance  = np.cov(df , rowvar=False)

# Covariance matrix power of -1
covariance_pm1 = np.linalg.matrix_power(covariance, -1)

# Center point
centerpoint = np.mean(df , axis=0)

In [ ]:
distances = []
for i, val in enumerate(df):
      p1 = val
      p2 = centerpoint
      distance = (p1-p2).T.dot(covariance_pm1).dot(p1-p2)
      distances.append(distance)
distances = np.array(distances)

# Cutoff (threshold) value from Chi-Sqaure Distribution for detecting outliers 
cutoff = chi2.ppf(0.95, df.shape[1])

# Index of outliers
outlierIndexes = np.where(distances > cutoff )

print('--- Index of Outliers ----')
print(outlierIndexes)
# array([24, 35, 67, 81])

print('--- Observations found as outlier -----')
print(df[ distances > cutoff , :])

In [ ]:
## Finding ellipse dimensions 
pearson = covariance[0, 1]/np.sqrt(covariance[0, 0] * covariance[1, 1])
ell_radius_x = np.sqrt(1 + pearson)
ell_radius_y = np.sqrt(1 - pearson)
lambda_, v = np.linalg.eig(covariance)
lambda_ = np.sqrt(lambda_)

# Ellipse patch
ellipse = patches.Ellipse(xy=(centerpoint[0], centerpoint[1]),
                  width=lambda_[0]*np.sqrt(cutoff)*2, height=lambda_[1]*np.sqrt(cutoff)*2,
                  angle=np.rad2deg(np.arccos(v[0, 0])), edgecolor='#fab1a0')
ellipse.set_facecolor('#0984e3')
ellipse.set_alpha(0.5)
fig = plt.figure()
ax = plt.subplot()
ax.add_artist(ellipse)
plt.scatter(df1.iloc[: , 0], df1.iloc[ : , 1], c = df1.iloc[ : , 3])
plt.colorbar()
# plt.scatter(df[outlierIndexes[0] , 0],df[outlierIndexes[1] , 1], color=)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')


p = ax.scatter(df1.iloc[: , 0], df1.iloc[ : , 1], df1.iloc[ : , 2],c = df1.iloc[ : , 3])
ax.set_xlabel('Ent temp(C)')
ax.set_ylabel('Exit temp(C)')
ax.set_zlabel('Motor temp(C)')
fig.colorbar(p, pad=0.15)
plt.show()